### Connect to workspace

In [5]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

1.0.39


In [6]:
ws = Workspace.from_config()

In [7]:
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

prodmod	westus2	prodmod_rg	westus2


### Upload data

In [14]:
ds = ws.get_default_datastore()

In [15]:
ds.upload(src_dir='../data', target_path='bf_data', overwrite=False, show_progress=True)

Uploading ../data\test.csv
Uploading ../data\train.csv
Uploading ../data\training_data.csv
Uploaded ../data\test.csv, 1 files out of an estimated total of 3
Uploaded ../data\train.csv, 2 files out of an estimated total of 3
Uploaded ../data\training_data.csv, 3 files out of an estimated total of 3


$AZUREML_DATAREFERENCE_2ab1f20d64b84545bc5a0860673e4aea

### Create an experiment

In [9]:
experiment_name = 'keras-black-friday'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

### Create Compute

In [11]:
compute_target = ws.compute_targets['prodmod-compute1']

In [12]:
from azureml.train.dnn import TensorFlow

In [13]:
script_folder = '.'

In [31]:
script_params = {
    '--data-folder': ds.path('bf_data').as_mount(),
    '--batch-size': 512,
    '--first-layer-neurons': 512,
    '--second-layer-neurons': 256,
    '--learning-rate': 0.01
}

In [32]:
estimator = TensorFlow(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                pip_packages=['keras', 'pandas', 'numpy', 'sklearn'],
                entry_script='train.py',
                use_gpu=False)

framework_version is not specified, defaulting to version 1.13.


In [33]:
run = exp.submit(estimator)

In [34]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [35]:
model = run.register_model(model_name='keras_bf', model_path='outputs/model')